In [15]:
%load_ext autoreload
%autoreload 2

from datasets.kaggle import KaggleDataset
import pandas as pd

dataset = KaggleDataset(timescale="Daily")

dataset.df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,close,date
2650,376.28,2014-11-28
2649,376.72,2014-11-29
2648,373.34,2014-11-30
2647,378.39,2014-12-01
2646,379.25,2014-12-02
...,...,...
4,39231.64,2022-02-25
3,39146.66,2022-02-26
2,37712.68,2022-02-27
1,43178.98,2022-02-28


In [16]:
# Add this code to a new cell in the notebook

import plotly.graph_objects as go
from signals import sma_filter, lma_filter, ema_filter, wma_signal, complex_signal

def plot_signals_with_wma(data: pd.DataFrame, window_size=20):
    """
    Plots the price along with EMA, SMA, LMA, and complex signals using the wma function.
    
    :param data: Pandas DataFrame containing 'date' and 'close' columns.
    :param window_size: Window size for the moving averages.
    """
    # Extract price series
    price = data['close'].values
    date = data['date'].values

    # Calculate SMA, EMA, and LMA using the wma function
    sma = wma_signal(price , sma_filter(window_size))
    ema = wma_signal(price , ema_filter(window_size, 2 / (window_size + 1)))
    lma = wma_signal(price, lma_filter(window_size))

    # Calculate complex signal
    complex_sig = complex_signal(price, 0.5, 0, 0.5, window_size, window_size, window_size, 2 / (window_size + 1))

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=date, y=price, mode='lines+markers', name='Price', line_shape='spline'))
    fig.add_trace(go.Scatter(x=date, y=sma, mode='lines', name='SMA', line_shape='spline', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=date, y=ema, mode='lines', name='EMA', line_shape='spline', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=date, y=lma, mode='lines', name='LMA', line_shape='spline', line=dict(color='orange')))
    # fig.add_trace(go.Scatter(x=date, y=complex_sig, mode='lines', name='Complex Signal', line_shape='spline'))

    fig.update_layout(
        title="Price and Moving Averages (Using WMA)",
        xaxis_title="Date",
        yaxis_title="Price",
        legend_title="Signals",
        template="plotly_dark"
    )

    fig.show()

# Example usage:
dataset = KaggleDataset(timescale="Daily")
# data = dataset.df
plot_signals_with_wma(dataset.df, window_size=10)

The above plot is interactive, you can zoom in to certain regions of the graph.

In [17]:
from signals import buysell_signals

def plot_signals_with_difference(data: pd.DataFrame):
    price, date = data['close'].values, data['date'].values

    short_filter = sma_filter(10)
    long_filter = sma_filter(20)
    short_signal = wma_signal(price, short_filter)
    long_signal = wma_signal(price, long_filter)
    buy_sell_signal = buysell_signals(short_signal, long_signal)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=date, y=price, mode='lines+markers', name='Price', line_shape='spline'))
    fig.add_trace(go.Scatter(x=date, y=short_signal, mode='lines', name='Short Signal', line_shape='spline'))
    fig.add_trace(go.Scatter(x=date, y=long_signal, mode='lines', name='Long Signal', line_shape='spline'))
    fig.add_trace(go.Scatter(x=date, y=buy_sell_signal, yaxis="y2", mode='lines', name='Buy/Sell Signal'))

    fig.update_layout(
        title="Price and Difference Signals",
        xaxis_title="Date",
        yaxis_title="Price",
        yaxis2=dict(title="Difference Signal", overlaying='y', side='right', fixedrange=True),
        legend_title="Signals",
        template="plotly_dark"
    )

    fig.show()

dataset = KaggleDataset(timescale="Daily")
plot_signals_with_difference(dataset.df)